In [ ]:
import random
from prettytable import PrettyTable
#!pip install fpdf
from fpdf import FPDF
import json
import csv
import os
from IPython.display import Image, display
from IPython.display import display, HTML
from termcolor import colored


def match_keywords_to_inventory(preprocessed_data, inventory):
    # Dummy matching logic
    matched_items = [
        item
        for item in inventory
        if (
            any(keyword.lower() in item["category"].lower() for keyword in preprocessed_data["preferred_categories"])
            and ((item["gender"].lower() == preprocessed_data["gender"].lower()) or (item["gender"].lower() == "u"))
            and (item["price"] <= preprocessed_data["budget"])
            and (item["stock_count"] > 0)  # Consider items with a positive stock count
        )
    ]
    return matched_items

def process_sale():
    user_name = input("Name: ")
    mobile_number = input("Mobile Number: ")
    occasion = "Birthday"
    age = 30
    gender = input("Gender (M/F): ")
    address = input("Enter your address: ")
    print("Categories:")
    print("1. Clothing")
    print("2. Jewelry")
    print("3. Home Decor")
    selected_categories = input("Select preferred categories (comma-separated numbers): ").split(",")
    preferred_categories = []

    for category_num in selected_categories:
        if category_num == "1":
            preferred_categories.append("Clothing")
        elif category_num == "2":
            preferred_categories.append("Jewelry")
        elif category_num == "3":
            preferred_categories.append("Home Decor")

    budget = int(input("Budget: "))

    # Read inventory from CSV
    inventory = []
    with open("inventory.csv", newline='') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            inventory.append({
                "item": row["item"],
                "price": float(row["price"]),
                "category": row["category"],
                "gender": row["gender"],
                "stock_count": int(row["stock_count"]),
            })

    # Preprocess user inputs
    processed_data = {
        "user_name": f"{user_name} - {mobile_number}",
        "occasion": occasion,
        "age": age,
        "gender": gender,
        "preferred_categories": preferred_categories,
        "budget": budget,
    }

    # Match keywords, gender, budget, and consider stock count
    matched_items = match_keywords_to_inventory(processed_data, inventory)
    random.shuffle(matched_items)

    # Display randomized recommendations to the user within the budget
    print("\n")
    print("Thank you for choosing giftaSPAC")
    print(f"Hello {user_name}, here are your personalized randomized recommendations within your budget:")

    order_number = 1
    try:
        with open("order_details.csv", "r", newline='') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                order_number = int(row["order_number"]) + 1
    except (FileNotFoundError, ValueError):
        pass  # If the file doesn't exist, start with order_number = 1

    if matched_items:
        total_price = 0
        table = PrettyTable(["Item", "Price", "Category", "Gender", "Stock Count"])
        ordered_items = []
        for item in matched_items:
            if total_price + item["price"] <= processed_data["budget"] and item["stock_count"] > 0:
                table.add_row([item["item"], item["price"], item["category"], item["gender"], item["stock_count"]])
                total_price += item["price"]
                ordered_items.append(item)

        if total_price == 0:
            print("Cannot predict within budget. Please adjust your preferences or budget.")
        else:
            print(table)
            print("Total price as per products:", total_price)
            print("Profit: ", budget - total_price)

            # Ask the user to approve the order
            user_approval = input("Do you approve the order? (Y/N): ").upper()

            if user_approval == "Y":
                for ordered_item in ordered_items:
                    for inv_item in inventory:
                        if inv_item["item"] == ordered_item["item"]:
                            inv_item["stock_count"] -= 1

                # Save the updated inventory to inventory.json and csv
                with open("inventory.json", "w") as json_file:
                    json.dump(inventory, json_file, indent=2)
                with open("inventory.csv", "w", newline='') as csv_file:
                    fieldnames = ["item", "price", "category", "gender", "stock_count"]
                    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                    csv_writer.writeheader()
                    csv_writer.writerows(inventory)

                # Generate a bill as a PDF
                pdf = FPDF()
                pdf.add_page()
                pdf.set_font("Arial", size=12)
                pdf.image("https://abhinavchebrolu.co.in/SPAC/images/Logo%20red.png", x=10, y=8, w=33)

                pdf.set_font("Times", size=14)

                pdf.cell(200, 10, txt=f"Tax Invoice/Bill of Supply - Order Confirmation \t\t\t", ln=True, align="R")

                pdf.set_font("Arial", "B", size=12)
                pdf.cell(200, 5, txt=f"Order Number: SPAC000{order_number}\t\t\t", ln=True, align="R")
                pdf.ln(9)
                pdf.cell(200, 5, txt=f"CustName: {user_name}", ln=True)
                pdf.cell(200, 5, txt=f"Mobile: {mobile_number}", ln=True)
                pdf.cell(200, 5, txt=f"CustAddress: {address}", ln=True)
                pdf.cell(200, 5, txt=f"Occasion: Happy {occasion}", ln=True)
                pdf.ln(7)

                pdf.set_font("Arial", size=12)

                # Adding the table to the PDF
                pdf.cell(200, 5, txt="Here you go, SPAC's Recommended Products:", ln=True)
                pdf.ln(5)

                # Set column widths for the table
                col_widths = [80, 25, 35, 20, 30]

                # Set background color for the first row
                pdf.set_fill_color(192, 192, 192)  # RGB values for grey color
                pdf.set_font("Arial", "B", size=12)
                # Add table headers with background color
                for i in range(len(table.field_names)):
                    pdf.cell(col_widths[i], 10, txt=str(table.field_names[i]), border=1, align="C", fill=True)
                pdf.ln()

                # Reset background color for subsequent rows
                pdf.set_fill_color(255, 255, 255)  # Reset to white background
                pdf.set_font("Arial", size=12)
                # Add table rows
                for row in table._rows:
                    for i in range(len(row)):
                        pdf.cell(col_widths[i], 10, txt=str(row[i]), border=1)
                    pdf.ln()

                pdf.ln(10)
                pdf.cell(200, 5, txt=f"Total Price: {total_price}", ln=True)
                pdf.cell(200, 5, txt="Thank you for choosing giftaSPAC!", ln=True)

                pdf.output(f"order_confirmation_SPAC000{order_number}.pdf")
                order_details = {
                    "order_number": order_number,
                    "user_name": user_name,
                    "mobile": mobile_number,
                    "address": address,
                    "gender": gender,
                    "occasion": occasion,
                    "items": [
                        {
                            "item": item["item"],
                            "price": item["price"],
                            "category": item["category"],
                            "gender": item["gender"],
                        }
                        for item in ordered_items
                    ],
                    "total_price": total_price,
                }

                with open("order_details.json", "a") as order_file:
                    json.dump(order_details, order_file, indent=2)
                    order_file.write("\n")

                fieldnames = ["order_number", "user_name", "mobile", "address", "gender", "occasion", "items", "total_price"]
                ordered_items_str = ', '.join([f"{item['item']} - {item['price']} INR" for item in ordered_items])
                with open("order_details.csv", "a", newline='') as order_file:
                    csv_writer = csv.DictWriter(order_file, fieldnames=fieldnames)

                    # Check if the file is empty and write the header
                    if os.stat("order_details.csv").st_size == 0:
                        csv_writer.writeheader()

                    csv_writer.writerow({
                        "order_number": order_number,
                        "user_name": user_name,
                        "mobile": mobile_number,
                        "address": address,
                        "gender": gender,
                        "occasion": occasion,
                        "items": ordered_items,
                        "total_price": total_price
                    })

                print(f"Order confirmed! Order details have been saved in order_confirmation_SPAC000{order_number}.pdf and order_details.json.")
                order_number += 1
            else:
                print("Order canceled.")
    else:
        print("Cannot predict within budget. Please adjust your preferences or budget.")

def check_order():
    print("Options:")
    print("1. Check order by order number")
    print("2. Check order by mobile number")
    option = input("Select an option (1 or 2): ")

    if option == "1":
        order_number = input("Enter the order number: ")
        check_order_by_order_number(order_number)
    elif option == "2":
        mobile_number = input("Enter the mobile number: ")
        check_order_by_mobile_number(mobile_number)
    else:
        print("Invalid option. Please select 1 or 2.")

def display_order_items(items_str):
    # Display order items with columns separated by "-"
    items = items_str.split(', ')
    if items_str and items:
        header = ["Item", "Price", "Category", "Gender"]
        print("-".join(header))
        for item_str in items:
            item = item_str.split("-")
            print("-".join(item))
    else:
        print("Invalid or empty 'items' field in the order details.")

# Modify the check_order_by_order_number and check_order_by_mobile_number functions:

def check_order_by_order_number(order_number):
    with open("order_details.csv", "r", newline='') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row["order_number"] == order_number:
                print("---------------------------")
                print("Order details:")
                print(f"Order Number: {row['order_number']}")
                print(f"Customer Name: {row['user_name']}")
                print(f"Mobile Number: {row['mobile']}")
                print(f"Address: {row['address']}")
                print(f"Gender: {row['gender']}")
                print(f"Occasion: {row['occasion']}")
                display_order_items(row['items'])
                print(f"Total Price: {row['total_price']}")
                return
        print(f"No order found with order number {order_number}.")

def check_order_by_mobile_number(mobile_number):
    with open("order_details.csv", "r", newline='') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row["mobile"] == mobile_number:
                print("---------------------------")
                print("Order details:")
                print(f"Order Number: {row['order_number']}")
                print(f"Customer Name: {row['user_name']}")
                print(f"Mobile Number: {row['mobile']}")
                print(f"Address: {row['address']}")
                print(f"Gender: {row['gender']}")
                print(f"Occasion: {row['occasion']}")
                display_order_items(row['items'])
                print(f"Total Price: {row['total_price']}")
                return
        print(f"No order found with mobile number {mobile_number}.")

def summary(inventory, orders):
    total_inventory_items = sum(item["stock_count"] for item in inventory)
    total_sales_amount = sum(order["total_price"] for order in orders)
    total_orders_count = len(orders)-1

    print("\nSummary:")
    display_colored_box("Total Inventory Items", total_inventory_items, color="green")
    display_colored_box("Total Sales Amount", total_sales_amount, color="blue")
    display_colored_box("Total Orders Count", total_orders_count, color="orange")

def display_colored_box(label, value, color="white"):
    colored_label = colored(f"{label}:", color="white", on_color=color, attrs=['bold'])
    colored_value = colored(value, color="white", on_color=color, attrs=['bold'])

    # Choose a background color for the box
    background_color = "#f0f0f0"  # You can change this hex code to your desired color

    # Adjust the width by changing the value in style="width: 150px;"
    html_code = f'<div style="border: 2px solid {color}; border-radius: 5px; padding: 5px; margin: 5px; background-color: {background_color}; width: 300px;"><b>{colored_label}</b> {colored_value}</div>'
    display(HTML(html_code))




def load_inventory():
    inventory = []
    with open("inventory.csv", newline='') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            inventory.append({
                "item": row["item"],
                "price": float(row["price"]),
                "category": row["category"],
                "gender": row["gender"],
                "stock_count": int(row["stock_count"]),
            })
    return inventory

def read_orders():
    orders = []
    try:
        with open("order_details.csv", "r", newline='') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                items_str = row["items"]
                if items_str:
                    try:
                        items = json.loads(items_str)
                    except json.JSONDecodeError:
                        items = []
                else:
                    items = []
                orders.append({
                    "order_number": int(row["order_number"]),
                    "user_name": row["user_name"],
                    "mobile": row["mobile"],
                    "address": row["address"],
                    "gender": row["gender"],
                    "occasion": row["occasion"],
                    "items": items,
                    "total_price": float(row["total_price"]),
                })
    except FileNotFoundError:
        pass  # If the file doesn't exist, return an empty list
    return orders




def main():
    while True:
        html_code = """
        <div style="display: flex; height: 100px; width: 162.5px;">
            <img src="https://abhinavchebrolu.co.in/SPAC/images/Logo%20red.png" alt="Logo" style="width: 100%; height: 100%;">
        </div>
        """

        # Display the HTML code
        display(HTML(html_code))

        # Load inventory and orders
        inventory = load_inventory()
        orders = read_orders()

        # Display summary
        summary(inventory, orders)

        print("\nOptions:")
        print("1. Point of Sale")
        print("2. Check Order Details")
        print("3. Exit")
        choice = input("Select an option (1, 2, or 3): ")

        if choice == "1":
            process_sale()
            break
        elif choice == "2":
            check_order()
            break
        elif choice == "3":
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid option. Please select 1, 2, or 3.")

if __name__ == "__main__":
    main()



Summary:



Options:
1. Point of Sale
2. Check Order Details
3. Exit
